In [1]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('../fusion'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
from preprocessing_datasets.preprocessing_book import *

In [3]:
start_time = time.time()
data = clean_book()
data

,index,ISBN_10,seller_link,big_cate,author,dirtyAuthor,oldAuthor,title
0,0,0000219118,ground-zero-books%2c-ltd-silver-spring/62893/sf,history-books,"[Association for Asian Studies, Inc]","[association for asian studies, inc]",asian association for inc studies,"The Journal of Asian Studies, Volume 47, Numbe..."
1,1,0000219118,pursuit-of-happiness-books-oakland-ca/81375/sf,history-books,"[The Association for Asian Studies, Inc]","[the association for asian studies, inc]",asian association for inc studies the,The Journal of Asian Studies
2,2,0000788589,archway-books-mana/685127/sf,science-books,"[Pocknall, D. T. And D. C. Mildenhall, Pocknal...","[pocknall, d. t., d. c. mildenhall]",d pocknall t,Late Oligocene - Early Miocene Spores and Poll...
3,2,0000788589,archway-books-mana/685127/sf,science-books,"[Pocknall, D. T. And D. C. Mildenhall, Pocknal...","[pocknall, d. t., d. c. mildenhall]",c d mildenhall,Late Oligocene - Early Miocene Spores and Poll...
4,3,0000788589,arty-bees-books-wellington/6226172/sf,science-books,"[Buckeridge, John Stewart]","[buckeridge, john stewart]",buckeridge john stewart,Fossil Barnacles (Cirripedia: Thoracica) of Ne...
...,...,...,...,...,...,...,...,...
2199026,1891013,9994533150,mark-henderson-olathe-ks-u.s.a/2095040/sf,literature,"[Tanner, Clara Lee]","[tanner, clara lee]",clara lee tanner,Indian Arts and Crafts
2199027,1891014,9994533150,owlsbooks-toledo-md-u.s.a/51218811/sf,literature,[Clara Lee Tanner],[clara lee tanner],clara lee tanner,Indian Arts and Crafts
2199028,1891015,9994533150,second-city-books-aurora-il-u.s.a/63477172/sf,literature,"[Tanner, Clara Lee]","[tanner, clara lee]",clara lee tanner,Indian Arts and Crafts
2199029,1891016,9998111552,dogstar-books-lancaster-pa-u.s.a/50869000/sf,history-books,"[Nixon, Richard M.]","[nixon, richard m.]",m nixon richard,RN; The Memoirs of Richard Nixon


In [4]:
start_time = time.time()
data = clean_book()
data.drop_duplicates(['ISBN_10', 'big_cate', 'seller_link'], keep="first", inplace=True)
data['numberOfAuthors'] = data['authors'].map(lambda x: len(ValueUtils.split_values(x)))
data['dirtyAuthor'] = data['authors'].map(lambda x: ValueUtils.split_values(x))
data = data.explode('dirtyAuthor')
data['oldAuthor'] = data['dirtyAuthor'].map(lambda x: ValueUtils.clean_value(x))
print("TIME: {}".format(time.time() - start_time))

TIME: 104.16798281669617


In [6]:
start_time = time.time()
data = data.groupby(['ISBN_10', 'seller_link', 'big_cate'], group_keys=False).agg(author=('authors', list), dirtyAuthor=('dirtyAuthor', list), oldAuthor=('oldAuthor', list),title=('title', 'first')).reset_index()
print("TIME: {}".format(time.time() - start_time))
#data

TIME: 514.3906879425049


In [7]:
start_time = time.time()
data['oldAuthor'] = data['oldAuthor'].map(lambda x: ValueUtils.retain_only_values_with_alphabet(x))
data['oldAuthor'] = data['oldAuthor'].map(lambda x: ValueUtils.retain_only_short_known_values(x))
data['oldAuthor'] = data['oldAuthor'].map(lambda x: list(x))
data = data.explode('oldAuthor')
data = data.dropna()
data = data.reset_index()
print("TIME: {}".format(time.time() - start_time))

TIME: 20.793272018432617


In [8]:
data

,index,ISBN_10,seller_link,big_cate,author,dirtyAuthor,oldAuthor,title
0,0,0000219118,ground-zero-books%2c-ltd-silver-spring/62893/sf,history-books,"[Association for Asian Studies, Inc]","[association for asian studies, inc]",asian association for inc studies,"The Journal of Asian Studies, Volume 47, Numbe..."
1,1,0000219118,pursuit-of-happiness-books-oakland-ca/81375/sf,history-books,"[The Association for Asian Studies, Inc]","[the association for asian studies, inc]",asian association for inc studies the,The Journal of Asian Studies
2,2,0000788589,archway-books-mana/685127/sf,science-books,"[Pocknall, D. T. And D. C. Mildenhall, Pocknal...","[pocknall, d. t., d. c. mildenhall]",c d mildenhall,Late Oligocene - Early Miocene Spores and Poll...
3,2,0000788589,archway-books-mana/685127/sf,science-books,"[Pocknall, D. T. And D. C. Mildenhall, Pocknal...","[pocknall, d. t., d. c. mildenhall]",d pocknall t,Late Oligocene - Early Miocene Spores and Poll...
4,3,0000788589,arty-bees-books-wellington/6226172/sf,science-books,"[Buckeridge, John Stewart]","[buckeridge, john stewart]",buckeridge john stewart,Fossil Barnacles (Cirripedia: Thoracica) of Ne...
...,...,...,...,...,...,...,...,...
2199026,1891013,9994533150,mark-henderson-olathe-ks-u.s.a/2095040/sf,literature,"[Tanner, Clara Lee]","[tanner, clara lee]",clara lee tanner,Indian Arts and Crafts
2199027,1891014,9994533150,owlsbooks-toledo-md-u.s.a/51218811/sf,literature,[Clara Lee Tanner],[clara lee tanner],clara lee tanner,Indian Arts and Crafts
2199028,1891015,9994533150,second-city-books-aurora-il-u.s.a/63477172/sf,literature,"[Tanner, Clara Lee]","[tanner, clara lee]",clara lee tanner,Indian Arts and Crafts
2199029,1891016,9998111552,dogstar-books-lancaster-pa-u.s.a/50869000/sf,history-books,"[Nixon, Richard M.]","[nixon, richard m.]",m nixon richard,RN; The Memoirs of Richard Nixon


In [9]:
type(data)

pandas.core.frame.DataFrame

In [11]:
data.to_csv('books_cleaned.csv', index=False)

In [12]:
data

,index,ISBN_10,seller_link,big_cate,author,dirtyAuthor,oldAuthor,title
0,0,0000219118,ground-zero-books%2c-ltd-silver-spring/62893/sf,history-books,"[Association for Asian Studies, Inc]","[association for asian studies, inc]",asian association for inc studies,"The Journal of Asian Studies, Volume 47, Numbe..."
1,1,0000219118,pursuit-of-happiness-books-oakland-ca/81375/sf,history-books,"[The Association for Asian Studies, Inc]","[the association for asian studies, inc]",asian association for inc studies the,The Journal of Asian Studies
2,2,0000788589,archway-books-mana/685127/sf,science-books,"[Pocknall, D. T. And D. C. Mildenhall, Pocknal...","[pocknall, d. t., d. c. mildenhall]",c d mildenhall,Late Oligocene - Early Miocene Spores and Poll...
3,2,0000788589,archway-books-mana/685127/sf,science-books,"[Pocknall, D. T. And D. C. Mildenhall, Pocknal...","[pocknall, d. t., d. c. mildenhall]",d pocknall t,Late Oligocene - Early Miocene Spores and Poll...
4,3,0000788589,arty-bees-books-wellington/6226172/sf,science-books,"[Buckeridge, John Stewart]","[buckeridge, john stewart]",buckeridge john stewart,Fossil Barnacles (Cirripedia: Thoracica) of Ne...
...,...,...,...,...,...,...,...,...
2199026,1891013,9994533150,mark-henderson-olathe-ks-u.s.a/2095040/sf,literature,"[Tanner, Clara Lee]","[tanner, clara lee]",clara lee tanner,Indian Arts and Crafts
2199027,1891014,9994533150,owlsbooks-toledo-md-u.s.a/51218811/sf,literature,[Clara Lee Tanner],[clara lee tanner],clara lee tanner,Indian Arts and Crafts
2199028,1891015,9994533150,second-city-books-aurora-il-u.s.a/63477172/sf,literature,"[Tanner, Clara Lee]","[tanner, clara lee]",clara lee tanner,Indian Arts and Crafts
2199029,1891016,9998111552,dogstar-books-lancaster-pa-u.s.a/50869000/sf,history-books,"[Nixon, Richard M.]","[nixon, richard m.]",m nixon richard,RN; The Memoirs of Richard Nixon
